## Preprocess `.mat` files and reorder the betas, then save


### Load data in notebook 
(we are dealing with `.mat` files) (**Remark.** note that we have 3 different types of data. `glm1, glm24, glm25`)

The repetition time (TR) is different for each `glm`. So for example, TR of `glm1` is one block, so 3x60s = 180s.

- The blocks (`glm 1`); TR = 180 s
- The levels (`glm 25`); TR = 60 s
- The 10 seconds boxcars (`glm 24`); TR = 10 s
    
    
from [here](https://stackoverflow.com/questions/874461/read-mat-files-in-python) and [scipy docu](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.io.loadmat.html)
```
Neither scipy.io.savemat, nor scipy.io.loadmat work for MATLAB arrays version 7.3. But the good part is that MATLAB version 7.3 files are hdf5 datasets. So they can be read using a number of tools, including NumPy.
```

- see `utils.py` for helper functions

### 2. Preprocess and reorder the rows in `B`, the average activity captured by the boxcar regressor for each block (the $\beta_i's$ ).

The bold signal is

$$
Y (\text{signal}) = \beta_1 x_1 + \beta_2 x_2 \dots + \beta_k x_k \\
  = \sum_i^k \beta_i x_i 
$$

### Data variables & structure 


<img src="http://drive.google.com/uc?export=view&id=1Dp27c1wmHMr0aNFHgBF9vZaCWFBpJGUI" style="height:230px"/>




In [6]:
import h5py
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import os 
import glob
import time
from copy import deepcopy
import numpy as np
import pandas as pd 

from nilearn import datasets
from nilearn import surface
from nilearn import plotting
from nilearn.input_data import NiftiMasker, NiftiLabelsMasker
import nibabel as nib

from brainiak import image, io
from brainiak.isc import isc, isfc, permutation_isc
from brainiak.isc import compute_summary_statistic
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d 
import seaborn as sns 
import pandas as pd
from importlib import reload 
import scipy.io as sio

# import own functions
import utils
from utils import decode_variable, get_in_shape_levels, get_in_shape_blocks
reload(utils)

%autosave 30
%matplotlib inline
sns.set(style = 'white', context='talk', font_scale=1, rc={"lines.linewidth": 2})

Autosaving every 30 seconds


# 1. Blocks

for blocks we take the `glm1` files

```Python
filename = 'beta_series_glm1_subj*_smooth.mat'
```

## 1.1 Load data in

In [7]:
num_subjects = 8

B_data_blocks = []
mask_data_blocks = []
Vmask_data_blocks = []
names_data_blocks = []

for i in range(num_subjects):
    idx = i+1
    
    # change filename to subject #
    data_dir = '/Users/Daphne/Desktop/beta_series_smooth/'
    filename = 'beta_series_glm1_subjk_smooth.mat'
    filename = filename.replace('k', str(idx))
    
    subject = h5py.File(data_dir+filename,'r') 
    #print(list(subject.keys()))
    print(f'Get data for subject {idx}')
    # load and save data for respective subject
    B = subject['B'].value
    mask = subject['mask'].value
    Vmask = subject['Vmask']
    
    # === decode level names ===
    names = decode_variable(data_dir+filename, 'names')
    
    # append to lists
    B_data_blocks.append(B)
    mask_data_blocks.append(mask)
    Vmask_data_blocks.append(Vmask)
    names_data_blocks.append(names)

Get data for subject 1
Get data for subject 2
Get data for subject 3
Get data for subject 4
Get data for subject 5
Get data for subject 6
Get data for subject 7
Get data for subject 8


In [8]:
print(names_data_blocks[0][0:10])

['Sn(1) vgfmri3_chase*bf(1)' 'Sn(1) vgfmri3_lemmings*bf(1)'
 'Sn(1) vgfmri3_bait*bf(1)' 'Sn(2) vgfmri3_plaqueAttack*bf(1)'
 'Sn(2) vgfmri3_helper*bf(1)' 'Sn(2) vgfmri3_zelda*bf(1)'
 'Sn(3) vgfmri3_lemmings*bf(1)' 'Sn(3) vgfmri3_plaqueAttack*bf(1)'
 'Sn(3) vgfmri3_zelda*bf(1)' 'Sn(4) vgfmri3_chase*bf(1)']


In [9]:
# mask_arr = mask_data_blocks

# path = '/Users/Daphne/Desktop/VGDL-fMRI-Python-Data-Analysis/Multivariate_analyses/data/'

# np.save(path+'mask_arr', mask_arr)

## 1.2 Preprocessing

- Reorder the data in `B` so that all blocks are in the same order.
- Get data in the right shape to perform ISC with brainiak.


In [10]:
ISC_arr_blocks = []
ordered_dfs_blocks = []

for s in range(num_subjects):
    
    print(f'Preprocess fMRI blocks data for subject {s+1}')
    # get the betas and game order from this
    B_s = B_data_blocks[s].T # transpose to get [blocks, voxels]
    names_s = names_data_blocks[s]
    
    dfOrdered, B_ordered = get_in_shape_blocks(B_s, names_s)
    
    ISC_arr_blocks.append(B_ordered)
    ordered_dfs_blocks.append(dfOrdered)
    
ISC_arr_blocks = np.array(ISC_arr_blocks)

Preprocess fMRI blocks data for subject 1
Preprocess fMRI blocks data for subject 2
Preprocess fMRI blocks data for subject 3
Preprocess fMRI blocks data for subject 4
Preprocess fMRI blocks data for subject 5
Preprocess fMRI blocks data for subject 6
Preprocess fMRI blocks data for subject 7
Preprocess fMRI blocks data for subject 8


In [11]:
ordered_dfs_blocks[2].head(10) # check order

,block,session,0,1,2,3,4,5,6,7,...,220065,220066,220067,220068,220069,220070,220071,220072,220073,220074
2,bait,Sn(1),-2.481648,-2.389143,-1.378340,-1.863973,-2.141596,-2.168410,-1.980433,-1.621671,...,-0.169477,-0.378513,-0.492044,-0.544052,-0.152289,-0.129932,0.275107,0.076450,-0.039035,0.007810
8,bait,Sn(3),-0.367010,-0.196446,-0.791404,-0.575174,-0.289704,0.028271,0.245217,0.260419,...,0.325280,0.148578,0.014657,-0.060109,0.007292,0.096778,0.420411,0.260720,0.147974,0.505072
14,bait,Sn(5),1.790470,1.536919,0.929763,1.562272,1.970997,1.969237,1.636931,1.190901,...,0.553769,0.426035,0.306009,0.209851,1.350008,1.332053,0.302795,0.288404,0.305053,1.622812
3,chase,Sn(2),-2.513420,-2.539430,-1.154479,-1.404383,-1.738660,-1.941170,-1.944731,-1.694107,...,0.301603,0.545929,0.868380,1.089191,0.263032,0.316593,0.540303,0.688182,0.920353,0.561565
7,chase,Sn(3),-1.036697,-0.917888,-0.651373,-0.770894,-0.810088,-0.718126,-0.540520,-0.364067,...,0.890840,1.101161,1.255906,1.303964,0.119302,0.052019,1.137229,1.390685,1.587178,0.525781
12,chase,Sn(5),-0.951518,-0.895393,-0.903024,-0.755501,-0.707465,-0.662530,-0.588037,-0.562128,...,0.970452,0.671771,0.324982,0.065867,0.462323,0.496855,0.936099,0.691683,0.407407,0.512947
1,helper,Sn(1),0.052872,0.128258,-0.524896,-0.408537,-0.000135,0.450675,0.646527,0.506248,...,0.481889,0.064717,-0.405451,-0.853344,-0.287999,-0.098856,0.838010,0.377683,-0.137316,0.385629
9,helper,Sn(4),-2.311759,-2.429004,-2.139832,-1.691737,-1.467166,-1.420989,-1.489437,-1.539786,...,0.463179,0.641903,0.790120,0.824531,0.208585,0.327628,0.813510,1.019502,1.155964,0.771025
17,helper,Sn(6),-1.562838,-1.694666,-0.946912,-0.825241,-0.777357,-0.831859,-0.959117,-1.106628,...,0.594220,0.768350,0.916759,1.025551,1.285331,0.968586,0.628030,0.729031,0.796998,1.051900
4,lemmings,Sn(2),-0.056267,-0.152029,0.025156,0.128017,0.047057,-0.072915,-0.189363,-0.226938,...,0.317969,0.500673,0.686621,0.808590,-0.093731,-0.179231,0.600261,0.659544,0.744191,0.119157


In [12]:
# get datat in the right shape to perform ISC with brainiak
# swap those axes!
blocks_ISC = np.swapaxes(ISC_arr_blocks, 0, 1) # need to get [TRs, voxels, subjects]
blocks_ISC = np.swapaxes(blocks_ISC, 1, 2)

blocks_ISC.shape

(18, 220075, 8)

# 2. Games

- Collapse the session numbers to obtain a matrix `6 (games) x voxels`
- For each 3 rows, add the values for each respective column and take the average

In [13]:
ordered_dfs_games = []
B_ordered_games = []

for i, df in enumerate(ordered_dfs_blocks):
    
    # average betas by game name (block)
    df_game = ordered_dfs_blocks[i].groupby('block').mean()
    ordered_dfs_games.append(df_game)
    
    B_games = df_game.values # convert to np array for ISC analysis
    B_ordered_games.append(B_games)
    print(df_game.shape)

B_ordered_games = np.array(B_ordered_games)

(6, 220075)
(6, 220075)
(6, 220075)
(6, 220075)
(6, 220075)
(6, 220075)
(6, 220075)
(6, 220075)


In [14]:
B_ordered_games.shape

(8, 6, 220075)

In [15]:
# get datat in the right shape to perform ISC with brainiak
# swap those axes!
games_ISC = np.swapaxes(B_ordered_games, 0, 1) # need to get [TRs, voxels, subjects]
games_ISC = np.swapaxes(games_ISC, 1, 2)

games_ISC.shape

(6, 220075, 8)

In [16]:
games_ISC[0]

array([[ 0.87387085, -0.73672632, -0.35272933, ..., -0.50960539,
        -1.28747742, -1.01653192],
       [ 0.78006113, -0.71179776, -0.34955696, ..., -0.51687793,
        -1.22466587, -1.13050979],
       [ 0.65952041, -0.34637908, -0.41332698, ..., -0.15840919,
        -1.26114808, -0.9080451 ],
       ...,
       [-0.35900998, -0.40345447,  0.20852445, ..., -0.48222077,
         1.42995558,  1.4721481 ],
       [-0.74783314, -0.52178536,  0.13799743, ..., -0.6416115 ,
         0.85561911,  1.28345553],
       [ 1.64073255,  0.82627211,  0.71189812, ..., -0.24329543,
         1.10470839,  0.94246493]])

In [17]:
#sio.savemat('test.mat', {'mydata': arr})

# 3. Levels

for levels we take the `glm25` files

```Python
filename = 'beta_series_glm25_subj*_smooth.mat'
```

## 3.1 Load in smooth levels data

In [19]:
num_subjects = 8

B_data_levels = []
mask_data_levels = []
Vmask_data_levels = []
names_data_levels = []

for i in range(num_subjects):
    idx = i+1
    
    # change filename to subject #
    data_dir = '/Users/Daphne/Desktop/beta_series_smooth/'
    filename = 'beta_series_glm25_subjk_smooth.mat'
    filename = filename.replace('k', str(idx))
    
    subject = h5py.File(data_dir+filename,'r') 
    #print(list(subject.keys()))
    print(f'Get data for subject {idx}')
    # load and save data for respective subject
    B = subject['B'].value
    mask = subject['mask'].value
    Vmask = subject['Vmask']
    
    # === decode level names ===
    names = decode_variable(data_dir+filename, 'names')
    
    # append to lists
    B_data_levels.append(B)
    mask_data_levels.append(mask)
    Vmask_data_levels.append(Vmask)
    names_data_levels.append(names)

Get data for subject 1
Get data for subject 2
Get data for subject 3
Get data for subject 4
Get data for subject 5
Get data for subject 6
Get data for subject 7
Get data for subject 8


In [20]:
names_data_levels[0][:10]

array(['Sn(1) vgfmri3_chase_run_1_block_1_instance_1*bf(1)',
       'Sn(1) vgfmri3_chase_run_1_block_1_instance_2*bf(1)',
       'Sn(1) vgfmri3_chase_run_1_block_1_instance_3*bf(1)',
       'Sn(1) vgfmri3_lemmings_run_1_block_2_instance_1*bf(1)',
       'Sn(1) vgfmri3_lemmings_run_1_block_2_instance_2*bf(1)',
       'Sn(1) vgfmri3_lemmings_run_1_block_2_instance_3*bf(1)',
       'Sn(1) vgfmri3_bait_run_1_block_3_instance_1*bf(1)',
       'Sn(1) vgfmri3_bait_run_1_block_3_instance_2*bf(1)',
       'Sn(1) vgfmri3_bait_run_1_block_3_instance_3*bf(1)',
       'Sn(2) vgfmri3_plaqueAttack_run_2_block_1_instance_1*bf(1)'],
      dtype='<U57')

In [21]:
B_data_levels[3].shape

(220075, 54)

## 3.2 Preprocess

In [22]:
ISC_arr_levels = []
ordered_dfs_levels = []
clean_names_arr = [] 

for s in range(num_subjects):
    
    print(f'Preprocess fMRI data for subject {s+1}')
    # get the betas and game order from this
    B_s = B_data_levels[s].T # transpose to get [blocks, voxels]
    names_s = names_data_levels[s]
    
    level_names, dfOrdered, B_ordered = get_in_shape_levels(B_s, names_s)
    
    ISC_arr_levels.append(B_ordered)
    ordered_dfs_levels.append(dfOrdered)
    clean_names_arr.append(level_names)
    
ISC_arr_levels = np.array(ISC_arr_levels)

Preprocess fMRI data for subject 1
Preprocess fMRI data for subject 2
Preprocess fMRI data for subject 3
Preprocess fMRI data for subject 4
Preprocess fMRI data for subject 5
Preprocess fMRI data for subject 6
Preprocess fMRI data for subject 7
Preprocess fMRI data for subject 8


In [28]:
ordered_dfs_levels[1].head(54) # check order

,game,session,instance,level,0,1,2,3,4,5,...,220065,220066,220067,220068,220069,220070,220071,220072,220073,220074
12,bait_run_2_block_2,Sn(2),1,1,-0.636170,-0.373854,-1.497341,-2.112460,-2.415012,-2.243853,...,-0.038629,-0.039316,-0.151938,-0.356248,0.780166,0.548450,-0.330269,-0.504245,-0.659055,0.951636
13,bait_run_2_block_2,Sn(2),2,2,0.942014,1.007454,0.715748,0.992761,1.298554,1.549959,...,-0.713421,-0.823860,-1.147676,-1.684841,-1.002751,-1.144127,-0.899558,-1.241139,-1.615280,-0.130938
14,bait_run_2_block_2,Sn(2),3,3,1.655500,1.412734,1.227655,1.776052,2.057731,2.052981,...,0.045613,0.033627,-0.303758,-0.969602,2.104960,1.762077,-0.022836,-0.258680,-0.718477,2.578229
27,bait_run_4_block_1,Sn(4),1,4,-2.147773,-2.092517,-2.035357,-2.470369,-2.851994,-3.012785,...,-0.865872,-0.742881,-0.551392,-0.220907,2.100336,0.926276,-0.463729,-0.270517,-0.053412,0.738639
28,bait_run_4_block_1,Sn(4),2,5,-0.270154,-0.262434,-1.101391,-1.208300,-1.261242,-1.228580,...,-1.226742,-1.247940,-1.448989,-1.703236,0.070426,-0.780465,-0.973810,-0.965997,-1.032061,-0.639038
29,bait_run_4_block_1,Sn(4),3,6,0.459462,0.474201,0.329827,0.373864,0.371919,0.445141,...,-0.553346,-0.429069,-0.532006,-0.864090,0.715096,-0.121409,-0.475154,-0.437522,-0.479703,0.283229
45,bait_run_6_block_1,Sn(6),1,7,0.957387,0.738800,2.399410,2.961179,3.211064,3.122286,...,-0.760641,-0.884861,-1.113301,-1.390764,1.085315,0.449789,-0.591991,-0.801478,-1.021741,0.871723
46,bait_run_6_block_1,Sn(6),2,8,2.382568,1.996827,4.545879,5.526391,5.973599,5.794201,...,-0.860787,-0.881726,-0.995501,-1.354120,0.865629,0.570306,-0.424392,-0.579131,-0.756057,0.949078
47,bait_run_6_block_1,Sn(6),3,9,1.276682,0.799029,2.344363,3.259416,3.691774,3.456368,...,-2.243427,-2.508078,-2.809407,-3.172503,-1.113008,-1.643137,-1.424382,-1.855104,-2.319343,-0.706388
3,chase_run_1_block_2,Sn(1),1,1,2.191652,1.856000,2.749278,2.857435,2.615516,2.246860,...,0.636880,1.031873,1.476854,1.983294,3.921660,2.658031,0.345488,0.719358,1.134821,2.226359


In [33]:
ordered_dfs_levels[1].head(5)

,game,session,instance,level,0,1,2,3,4,5,...,220065,220066,220067,220068,220069,220070,220071,220072,220073,220074
12,bait_run_2_block_2,Sn(2),1,1,-0.636170,-0.373854,-1.497341,-2.112460,-2.415012,-2.243853,...,-0.038629,-0.039316,-0.151938,-0.356248,0.780166,0.548450,-0.330269,-0.504245,-0.659055,0.951636
13,bait_run_2_block_2,Sn(2),2,2,0.942014,1.007454,0.715748,0.992761,1.298554,1.549959,...,-0.713421,-0.823860,-1.147676,-1.684841,-1.002751,-1.144127,-0.899558,-1.241139,-1.615280,-0.130938
14,bait_run_2_block_2,Sn(2),3,3,1.655500,1.412734,1.227655,1.776052,2.057731,2.052981,...,0.045613,0.033627,-0.303758,-0.969602,2.104960,1.762077,-0.022836,-0.258680,-0.718477,2.578229
27,bait_run_4_block_1,Sn(4),1,4,-2.147773,-2.092517,-2.035357,-2.470369,-2.851994,-3.012785,...,-0.865872,-0.742881,-0.551392,-0.220907,2.100336,0.926276,-0.463729,-0.270517,-0.053412,0.738639
28,bait_run_4_block_1,Sn(4),2,5,-0.270154,-0.262434,-1.101391,-1.208300,-1.261242,-1.228580,...,-1.226742,-1.247940,-1.448989,-1.703236,0.070426,-0.780465,-0.973810,-0.965997,-1.032061,-0.639038


In [34]:
ordered_dfs_levels[0].head(5)

,game,session,instance,level,0,1,2,3,4,5,...,220065,220066,220067,220068,220069,220070,220071,220072,220073,220074
6,bait_run_1_block_3,Sn(1),1,1,-0.371284,-0.226674,0.141564,-0.135850,-0.239876,-0.114735,...,-0.616665,-0.280196,0.282131,0.890054,4.087420,4.436308,-0.529103,-0.216954,0.217605,2.667310
7,bait_run_1_block_3,Sn(1),2,2,-0.675075,-0.548609,-0.093990,-0.466646,-0.650318,-0.597907,...,-0.431353,-0.214874,-0.070370,-0.316637,5.789201,6.352387,-0.133562,-0.006634,-0.018905,4.110570
8,bait_run_1_block_3,Sn(1),3,3,-0.804901,-0.642755,-0.725184,-1.012849,-1.169983,-1.110942,...,-0.088457,0.129538,-0.063181,-0.851084,4.041594,4.620839,-0.182671,-0.045232,-0.271131,2.716339
30,bait_run_4_block_2,Sn(4),1,4,1.118520,1.029883,0.637897,0.724377,0.618464,0.512915,...,0.880535,0.454054,-0.988663,-3.009454,1.063777,1.352335,0.895750,0.502366,-0.718263,2.426851
31,bait_run_4_block_2,Sn(4),2,5,0.972461,1.110638,0.350445,0.395563,0.424534,0.608492,...,0.627325,0.339463,-0.935144,-3.066810,4.146245,4.212489,0.885878,0.541749,-0.685535,5.860139


In [24]:
ISC_arr_levels.shape

(8, 54, 220075)

In [25]:
# get datat in the right shape to perform ISC with brainiak
# swap those axes!
levels_ISC = np.swapaxes(ISC_arr_levels, 0, 1) # need to get [TRs, voxels, subjects]
levels_ISC = np.swapaxes(levels_ISC, 1, 2)

levels_ISC.shape 

(54, 220075, 8)

In [35]:
# store dataframe for further reference
df = ordered_dfs_levels[1]

df.to_csv('ordered_betas_df_levels', encoding='utf-8', index=False)

In [15]:
# arr = levels_ISC

# np.save(path+'bold_data_levels', arr)

# 4. Boxcars (10 s)

for the Boxcars we take the `glm24` files

```Python
filename = 'beta_series_glm24_subj*_smooth.mat'
```

<font color=red>TODO: Boxcar data files are corrupted???</font> 

## 4.1 Load in smooth boxcar data

In [8]:
num_subjects = 8

B_data_boxcars = []
mask_data_boxcars = []
Vmask_data_boxcars = []
names_data_boxcars = []

for i in range(num_subjects):
    idx = i+1
    
    # change filename to subject #
    data_dir = '/Users/Daphne/Desktop/beta_series_smooth/'
    filename = 'beta_series_glm24_subjk_smooth.mat'
    filename = filename.replace('k', str(idx))
    
    if idx==4:
        print('Skipping sub 4, because is corrupted')
    
    else:
        subject = h5py.File(data_dir+filename,'r') 

        #print(list(subject.keys()))
        print(f'Get data for subject {idx}')
        # load and save data for respective subject
        B = subject['B'].value
        mask = subject['mask'].value
        Vmask = subject['Vmask']

        # === decode level names ===
        names = decode_variable(data_dir+filename, 'names')

        # append to lists
        B_data_boxcars.append(B)
        mask_data_boxcars.append(mask)
        Vmask_data_boxcars.append(Vmask)
        names_data_boxcars.append(names)

Get data for subject 1
Get data for subject 2
Get data for subject 3
Skipping sub 4, because is corrupted


OSError: Unable to open file (file signature not found)

In [5]:
os.path.isfile('/Users/Daphne/Desktop/beta_series_smooth/beta_series_glm24_subj4_smooth.mat')

True

In [ ]:
print(names_data_blocks[0][0:10])

# Quicklinks & Resources
    
- [Brainiak ISC documentation](https://brainiak.org/docs/brainiak.html#module-brainiak.isc)
- [Brainiak specific examples](https://github.com/brainiak/brainiak/tree/master/examples)